In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
from utils.data_ingestion_detached_house import (
    Dict,
    load_detached_house_data,
    calc_japan_detached_house_statistics,
    calc_pref_city_distribution,
    tokyo_wards,
)

from utils.japan_area_names import cities_jp_to_en, prefecture_dict_jp_to_en, epco_names
location_jp_to_en = {**cities_jp_to_en, **prefecture_dict_jp_to_en}
ftr_pref_muni = ["pref", "muni"]

In [4]:
# ============================================================================ #
# Detached house data
# ============================================================================ #

file_loc = "data/ref/e008_1.xlsx"

# load fine
df = load_detached_house_data(file_loc)	

# calculate the Japanese statistics
jp_dhouse_stat = calc_japan_detached_house_statistics(df)

# caculate the distribution on prefectures and cities
df_pref_distribution, df_city_distribution = calc_pref_city_distribution(df)

# DataFrame Index of the 21 cities
city_index_tuple = df_city_distribution.set_index(['pref', 'muni']).index.to_list()

In [5]:
# ============================================================================ #
# Municipal Demand from ENECHO (Monthly)
# ============================================================================ #

municipal_demand = pd.read_csv("data/ref/muni_demand_fy2022.csv")

demand_cities = municipal_demand[
    municipal_demand[["pref", "muni"]].apply(tuple, axis=1).isin(city_index_tuple)
]
demand_cities = demand_cities.set_index(['pref', 'muni'])
demand_tokyo = municipal_demand[municipal_demand['muni'].isin(tokyo_wards)].set_index('muni').drop('pref', axis=1)
demand_tokyo.index.name = None

demand_cities.loc[("東京都", "特別区部"), :] = demand_tokyo.sum()
demand_cities = demand_cities.astype(int)
demand_cities.rename(index=location_jp_to_en).reset_index()

,pref,muni,2022.4,2022.5,2022.6,2022.7,2022.8,2022.9,2022.10,2022.11,2022.12,2023.1,2023.2,2023.3,total
0,Hokkaido,Sapporo,723748,632649,562211,640183,645979,608037,599922,668017,821405,1006851,883062,776410,8568474
1,Miyagi,Sendai,536961,445147,451933,490997,507391,479034,428496,433848,537607,644690,601565,537866,6095535
2,Saitama,Saitama,432678,380648,407057,508913,555444,487621,403610,395128,454323,538039,541275,435168,5539904
3,Chiba,Chiba,393125,343364,355502,441599,493610,436540,380543,388847,424022,474402,482620,382477,4996651
4,Kanagawa,Yokohama,1308569,1184707,1256216,1495924,1655722,1535575,1297307,1232401,1348934,1561383,1573336,1341689,16791763
5,Kanagawa,Kawasaki,920480,870072,874173,972219,1040896,996377,897683,856831,920435,982253,980306,903233,11214964
6,Kanagawa,Sagamihara,295397,259043,277134,324994,348429,326205,282894,278917,297567,342132,349602,304380,3686694
7,Niigata,Niigata,341960,315702,297863,356165,381609,348302,294422,306807,358760,423083,407256,352335,4184264
8,Shizuoka,Shizuoka,308510,280246,279434,333554,363710,358089,303101,275973,290631,366630,337426,300034,3797338
9,Shizuoka,Hamamatsu,389076,348583,358173,422203,457699,462637,396960,352944,377578,466488,441504,401182,4875027


In [6]:
# ============================================================================ #
# EPCO Jurisdictions at the Municipal Level
# ============================================================================ #

epco_muni_juris = pd.read_csv("data/ref/epco_muni_jurisdiction.csv")
epco_muni_juris_dict = epco_muni_juris.set_index(ftr_pref_muni)['epco'].to_dict()

# assign epco for each municipality
municipal_demand['epco'] = municipal_demand[ftr_pref_muni].apply(
    lambda x: epco_muni_juris_dict.get(tuple(x)), axis=1
)

In [7]:
# ============================================================================ #
# EPCO level demand data based on the Municipal Data from ENECHO
# ============================================================================ #

municipal_demand_by_epco = municipal_demand.drop(ftr_pref_muni, axis=1).groupby("epco").sum().div(1_000).round(
    0
).sort_values("total", ascending=False)
municipal_demand_by_epco.index.name = None
municipal_demand_by_epco

,2022.4,2022.5,2022.6,2022.7,2022.8,2022.9,2022.10,2022.11,2022.12,2023.1,2023.2,2023.3,total
tokyo,21533.0,19307.0,20482.0,23914.0,25639.0,23756.0,20469.0,19851.0,21570.0,24663.0,24902.0,21373.0,267460.0
kansai,10774.0,9779.0,9837.0,11904.0,12867.0,12491.0,10614.0,9638.0,10425.0,12723.0,11953.0,11039.0,134044.0
chubu,9981.0,8998.0,9299.0,10725.0,11133.0,11240.0,9829.0,9164.0,9600.0,11324.0,10823.0,10112.0,122228.0
kyushu,6380.0,6167.0,6137.0,7237.0,8136.0,7579.0,6683.0,5963.0,6426.0,8148.0,7208.0,6684.0,82749.0
tohoku,6611.0,5840.0,5825.0,6393.0,6561.0,6252.0,5777.0,6013.0,6780.0,7855.0,7451.0,6685.0,78042.0
chugoku,4461.0,4027.0,4164.0,4825.0,5115.0,4917.0,4246.0,4004.0,4616.0,5298.0,4979.0,4587.0,55238.0
hokkaido,2405.0,2180.0,1960.0,2187.0,2211.0,2122.0,2120.0,2270.0,2626.0,3155.0,2762.0,2514.0,28512.0
hokuriku,2160.0,1984.0,2080.0,2315.0,2344.0,2216.0,2004.0,2031.0,2342.0,2659.0,2452.0,2275.0,26862.0
shikoku,2067.0,1948.0,2018.0,2349.0,2528.0,2301.0,2043.0,1882.0,2131.0,2570.0,2237.0,2190.0,26263.0
okinawa,560.0,594.0,640.0,793.0,891.0,875.0,761.0,625.0,577.0,599.0,524.0,497.0,7934.0


In [8]:
# ============================================================================ #
# Load Hourly Demand Data from EPCOs
# ============================================================================ #

epco_hourly_demand_fy2022 = pd.read_csv(
    "data/ref/epco_hourly_demand_fy2022.csv", index_col=0
)
epco_hourly_demand_fy2022.index = pd.to_datetime(epco_hourly_demand_fy2022.index)
epco_monthly_demand_fy2022 = (
    epco_hourly_demand_fy2022.resample("M")
    .sum()
    .div(1_000)
    .T.sort_values("2023-03-31", ascending=False)
)
epco_monthly_demand_fy2022.columns = municipal_demand.columns[2:14]

epco_monthly_demand_fy2022["total"] = epco_monthly_demand_fy2022.sum(axis=1)
epco_monthly_demand_fy2022

,2022.4,2022.5,2022.6,2022.7,2022.8,2022.9,2022.10,2022.11,2022.12,2023.1,2023.2,2023.3,total
tokyo,20693.4200,20358.7500,22623.1000,27104.5900,26746.5800,23161.8500,21046.9000,20896.6300,25385.8100,26709.0300,23492.9900,21429.5400,279649.1900
kansai,10166.1530,10141.4550,11340.0520,13434.5060,13735.6200,12005.6970,10472.8040,10331.2500,12809.4120,13252.4460,12083.1280,11054.6630,140827.1860
chubu,9777.4230,9507.5170,10701.7120,12076.6560,12107.8080,11388.2970,10002.2040,10014.3910,11849.8500,12072.1260,11044.7970,10395.2260,130938.0070
tohoku,6168.6910,5924.7030,6175.3640,6971.2490,6798.0810,6277.4340,6198.1870,6377.1110,7967.3400,8313.4610,7393.3130,6749.9440,81314.8780
kyushu,6066.2740,6095.2820,6896.2120,8250.8850,8388.9820,7079.3720,6275.0540,6086.6820,7919.1190,8022.3440,6861.5780,6591.6070,84533.3910
chugoku,4353.2070,4237.6920,4637.7940,5331.0200,5527.9580,4868.3780,4423.9060,4409.2830,5580.1300,5614.2820,4973.6850,4642.9580,58600.2930
hokkaido,2324.5780,2169.9930,2112.8650,2348.0250,2329.7890,2175.8500,2288.4880,2470.7650,3104.8360,3258.7950,2872.7730,2621.4750,30078.2320
hokuriku,2177.6900,2053.9680,2259.2640,2538.8940,2497.2400,2281.8080,2119.2770,2175.4320,2747.5470,2800.7080,2549.3990,2341.7090,28542.9360
shikoku,1967.8400,1923.2700,2200.4400,2540.0600,2644.1800,2266.1700,2081.3200,2043.8500,2549.9300,2619.9200,2332.1800,2166.1000,27335.2600
okinawa,598.8286,649.8255,778.3684,931.8275,940.4988,809.0525,713.1832,616.4287,592.7278,587.1513,511.8377,570.2972,8300.0272


In [9]:
# ============================================================================ #
# Calculate the transmission loss 
# assumption: transmission loss = ratio of the reported municipal demand (user side)
# and the EPCO demand (supply side)
# ============================================================================ #

epco_tx_loss = municipal_demand_by_epco['total']/epco_monthly_demand_fy2022['total']
epco_tx_loss

chubu       0.933480
chugoku     0.942623
hokkaido    0.947928
hokuriku    0.941109
kansai      0.951833
kyushu      0.978891
okinawa     0.955900
shikoku     0.960774
tohoku      0.959751
tokyo       0.956413
Name: total, dtype: float64

In [10]:
# ============================================================================ #
# Apply the transmission loss 
# ============================================================================ #

epco_hourly_demand_fy2022_scaled = epco_hourly_demand_fy2022*epco_tx_loss
epco_hourly_demand_fy2022_scaled.sum(axis=0)

chubu       122228000.0
chugoku      55238000.0
hokkaido     28512000.0
hokuriku     26862000.0
kansai      134044000.0
kyushu       82749000.0
okinawa       7934000.0
shikoku      26263000.0
tohoku       78042000.0
tokyo       267460000.0
dtype: float64

In [11]:
# ============================================================================ #
# Electricity Demand Share of each Municipality for their corresponding EPCO
# ============================================================================ #

municipal_demand_share = []
for epco, df_temp in municipal_demand.set_index(ftr_pref_muni).groupby('epco'):
	df_temp = df_temp.drop('epco',axis=1)/municipal_demand_by_epco.T[epco]/1000
	df_temp['epco'] = epco
	municipal_demand_share.append(df_temp)
municipal_demand_share = pd.concat(municipal_demand_share)
municipal_demand_share

2022.4    2022.5    2022.6    2022.7    2022.8    2022.9  \
pref muni                                                               
長野県  長野市   0.018874  0.017222  0.016155  0.016589  0.016589  0.015810   
     松本市   0.012046  0.011026  0.009960  0.009937  0.010179  0.009623   
     上田市   0.009178  0.008247  0.007919  0.007717  0.007472  0.007598   
     岡谷市   0.003188  0.002910  0.002791  0.002632  0.002515  0.002523   
     飯田市   0.005156  0.004674  0.004357  0.004291  0.004351  0.004317   
...             ...       ...       ...       ...       ...       ...   
静岡県  西伊豆町  0.000160  0.000164  0.000157  0.000149  0.000171  0.000185   
     函南町   0.000634  0.000634  0.000623  0.000608  0.000642  0.000658   
     清水町   0.000833  0.000805  0.000844  0.000819  0.000804  0.000858   
     長泉町   0.001085  0.001114  0.001179  0.001122  0.001012  0.001260   
     小山町   0.000905  0.000968  0.000956  0.000885  0.000869  0.000876   

            2022.10   2022.11   2022.12    2023.1    2023.2    2023.3  \
pref muni                                                               
長野県  長野市   0.016056  0.017773  0.018745  0.020304  0.020479  0.019461   
     松本市   0.010163  0.011462  0.012226  0.013555  0.013264  0.012441   
     上田市   0.007745  0.008635  0.009170  0.009761  0.009776  0.009465   
     岡谷市   0.002749  0.003123  0.003305  0.003411  0.003451  0.003281   
     飯田市   0.004483  0.004746  0.005039  0.005516  0.005447  0.005218   
...             ...       ...       ...       ...       ...       ...   
静岡県  西伊豆町  0.000179  0.000163  0.000150  0.000160  0.000156  0.000162   
     函南町   0.000636  0.000642  0.000643  0.000680  0.000661  0.000641   
     清水町   0.000880  0.000861  0.000810  0.000813  0.000823  0.000849   
     長泉町   0.001126  0.001073  0.001010  0.001030  0.001142  0.001140   
     小山町   0.000967  0.000999  0.000936  0.000898  0.000935  0.001033   

              total   epco  
pref muni                   
長野県  長野市   0.017859  chubu  
     松本市   0.011334  chubu  
     上田市   0.008559  chubu  
     岡谷市   0.002987  chubu  
     飯田市   0.004804  chubu  
...             ...    ...  
静岡県  西伊豆町  0.000163  tokyo  
     函南町   0.000643  tokyo  
     清水町   0.000832  tokyo  
     長泉町   0.001107  tokyo  
     小山町   0.000932  tokyo  

[1747 rows x 14 columns]

In [12]:
# ============================================================================ #
# Electricity Demand Share of the Main Cities for their corresponding EPCO
# ============================================================================ #

main_city_demand_share = municipal_demand_share[municipal_demand_share.index.isin(city_index_tuple)].copy()

df_temp = municipal_demand_share.copy()
df_temp = df_temp.reset_index()
df_temp = df_temp[df_temp["muni"].isin(tokyo_wards)].drop(['pref', 'muni', 'epco'], axis=1).sum()
df_temp['epco'] = 'tokyo'

main_city_demand_share.loc[("東京都", "特別区部"), :] = df_temp.copy()
main_city_demand_share.rename(index=location_jp_to_en)

2022.4    2022.5    2022.6    2022.7    2022.8  \
pref      muni                                                           
Shizuoka  Shizuoka    0.030910  0.031145  0.030050  0.031101  0.032670   
          Hamamatsu   0.038982  0.038740  0.038517  0.039366  0.041112   
Aichi     Nagoya      0.103663  0.106486  0.103011  0.111262  0.116717   
Okayama   Okayama     0.078301  0.078137  0.080160  0.083653  0.085180   
Hiroshima Hiroshima   0.107771  0.105193  0.107169  0.112940  0.115927   
Hokkaido  Sapporo     0.300935  0.290206  0.286842  0.292722  0.292166   
Kyoto     Kyoto       0.057807  0.055915  0.056126  0.059662  0.060228   
Osaka     Osaka       0.147092  0.150535  0.154268  0.158346  0.161946   
          Sakai       0.047048  0.048993  0.049343  0.046216  0.044996   
Hyogo     Kobe        0.068207  0.068115  0.068577  0.068132  0.069786   
Fukuoka   Kitakyushu  0.084395  0.087729  0.087038  0.082681  0.080648   
          Fukuoka     0.096623  0.096258  0.096847  0.103817  0.108153   
Kumamoto  Kumamoto    0.043855  0.043978  0.044299  0.046467  0.048439   
Miyagi    Sendai      0.081222  0.076224  0.077585  0.076802  0.077334   
Niigata   Niigata     0.051726  0.054059  0.051135  0.055712  0.058163   
Saitama   Saitama     0.020094  0.019716  0.019874  0.021281  0.021664   
Chiba     Chiba       0.018257  0.017784  0.017357  0.018466  0.019252   
Kanagawa  Yokohama    0.060770  0.061362  0.061333  0.062554  0.064578   
          Kawasaki    0.042747  0.045065  0.042680  0.040655  0.040598   
          Sagamihara  0.013718  0.013417  0.013531  0.013590  0.013590   
Tokyo     Tokyo       0.207373  0.209224  0.212013  0.219809  0.225555   

                        2022.9   2022.10   2022.11   2022.12    2023.1  \
pref      muni                                                           
Shizuoka  Shizuoka    0.031858  0.030837  0.030115  0.030274  0.032376   
          Hamamatsu   0.041160  0.040387  0.038514  0.039331  0.041195   
Aichi     Nagoya      0.113041  0.108245  0.100190  0.101488  0.111354   
Okayama   Okayama     0.083862  0.079093  0.078753  0.079456  0.083517   
Hiroshima Hiroshima   0.114961  0.107814  0.106019  0.112234  0.119632   
Hokkaido  Sapporo     0.286540  0.282982  0.294281  0.312797  0.319129   
Kyoto     Kyoto       0.059524  0.057357  0.057182  0.059353  0.062095   
Osaka     Osaka       0.160457  0.156126  0.149161  0.147583  0.146334   
          Sakai       0.045675  0.047860  0.048422  0.046139  0.043022   
Hyogo     Kobe        0.070355  0.069497  0.068296  0.068306  0.068993   
Fukuoka   Kitakyushu  0.081526  0.083289  0.083721  0.085633  0.082484   
          Fukuoka     0.103137  0.097160  0.094077  0.097670  0.103328   
Kumamoto  Kumamoto    0.047946  0.045599  0.043396  0.044597  0.048380   
Miyagi    Sendai      0.076621  0.074173  0.072152  0.079293  0.082074   
Niigata   Niigata     0.055710  0.050965  0.051024  0.052914  0.053862   
Saitama   Saitama     0.020526  0.019718  0.019905  0.021063  0.021816   
Chiba     Chiba       0.018376  0.018591  0.019588  0.019658  0.019235   
Kanagawa  Yokohama    0.064639  0.063379  0.062083  0.062538  0.063309   
          Kawasaki    0.041942  0.043856  0.043163  0.042672  0.039827   
          Sagamihara  0.013731  0.013821  0.014051  0.013795  0.013872   
Tokyo     Tokyo       0.220554  0.211549  0.206648  0.209744  0.211448   

                        2023.2    2023.3     total      epco  
pref      muni                                                
Shizuoka  Shizuoka    0.031177  0.029671  0.031068     chubu  
          Hamamatsu   0.040793  0.039674  0.039885     chubu  
Aichi     Nagoya      0.109750  0.106252  0.107942     chubu  
Okayama   Okayama     0.083666  0.080015  0.081346   chugoku  
Hiroshima Hiroshima   0.119616  0.112271  0.112215   chugoku  
Hokkaido  Sapporo     0.319718  0.308835  0.300522  hokkaido  
Kyoto     Kyoto       0.061600  0.058988  0.058985    kansai  
Osaka     Osaka       0.146771  0.148273  0.152439 

In [13]:
# ============================================================================ #
# Load the Prefectural Energy Electricity Share
# ============================================================================ #

df_pref_elec_share = pd.read_csv("data/ref/pref_electricity_share_2021.csv")
pref_elec_resi_share_dict = df_pref_elec_share.set_index('pref')['share_residential'].to_dict()
df_pref_elec_share.head(5)

,pref_id,pref,pref_en,industry,residential,total,share_residential
0,1,北海道,Hokkaido,17808.388412,10325.086932,28133.475344,0.367004
1,2,青森県,Aomori,5121.913207,2771.256394,7893.169601,0.351096
2,3,岩手県,Iwate,4809.258571,2395.013676,7204.272248,0.332444
3,4,宮城県,Miyagi,8565.021533,3629.233083,12194.254617,0.297618
4,5,秋田県,Akita,4728.322821,2153.935184,6882.258004,0.312969


In [14]:
# ============================================================================ #
# Calculate the Residential Share relative to the EPCO demand
# ============================================================================ #

main_city_epco_elec_resi_share = main_city_demand_share.copy()
main_city_epco_elec_resi_share = main_city_epco_elec_resi_share.reset_index().set_index(
    ftr_pref_muni + ["epco"]
)
for index, row in main_city_epco_elec_resi_share.iterrows():
	main_city_epco_elec_resi_share.loc[index] = main_city_epco_elec_resi_share.loc[index]*pref_elec_resi_share_dict[index[0]]
main_city_epco_elec_resi_share.rename(index=location_jp_to_en)

2022.4    2022.5    2022.6    2022.7  \
pref      muni       epco                                               
Shizuoka  Shizuoka   chubu     0.008533  0.008598  0.008295  0.008585   
          Hamamatsu  chubu     0.010761  0.010694  0.010633  0.010867   
Aichi     Nagoya     chubu     0.024660  0.025332  0.024505  0.026468   
Okayama   Okayama    chugoku   0.016558  0.016523  0.016951  0.017689   
Hiroshima Hiroshima  chugoku   0.026201  0.025575  0.026055  0.027458   
Hokkaido  Sapporo    hokkaido  0.110444  0.106507  0.105272  0.107430   
Kyoto     Kyoto      kansai    0.018907  0.018288  0.018357  0.019514   
Osaka     Osaka      kansai    0.045369  0.046431  0.047583  0.048840   
          Sakai      kansai    0.014512  0.015112  0.015219  0.014255   
Hyogo     Kobe       kansai    0.018541  0.018516  0.018642  0.018521   
Fukuoka   Kitakyushu kyushu    0.025701  0.026716  0.026506  0.025179   
          Fukuoka    kyushu    0.029425  0.029314  0.029493  0.031615   
Kumamoto  Kumamoto   kyushu    0.014189  0.014229  0.014333  0.015034   
Miyagi    Sendai     tohoku    0.024173  0.022686  0.023091  0.022858   
Niigata   Niigata    tohoku    0.013676  0.014292  0.013519  0.014729   
Saitama   Saitama    tokyo     0.006939  0.006808  0.006863  0.007349   
Chiba     Chiba      tokyo     0.004731  0.004609  0.004498  0.004786   
Kanagawa  Yokohama   tokyo     0.019768  0.019960  0.019951  0.020348   
          Kawasaki   tokyo     0.013905  0.014659  0.013883  0.013225   
          Sagamihara tokyo     0.004462  0.004364  0.004401  0.004421   
Tokyo     Tokyo      tokyo     0.060746  0.061288  0.062105  0.064389   

                                 2022.8    2022.9   2022.10   2022.11  \
pref      muni       epco                                               
Shizuoka  Shizuoka   chubu     0.009018  0.008794  0.008513  0.008313   
          Hamamatsu  chubu     0.011349  0.011362  0.011149  0.010632   
Aichi     Nagoya     chubu     0.027766  0.026891  0.025750  0.023834   
Okayama   Okayama    chugoku   0.018012  0.017734  0.016725  0.016653   
Hiroshima Hiroshima  chugoku   0.028184  0.027950  0.026212  0.025775   
Hokkaido  Sapporo    hokkaido  0.107226  0.105161  0.103855  0.108002   
Kyoto     Kyoto      kansai    0.019699  0.019469  0.018760  0.018702   
Osaka     Osaka      kansai    0.049951  0.049492  0.048156  0.046007   
          Sakai      kansai    0.013879  0.014088  0.014762  0.014936   
Hyogo     Kobe       kansai    0.018970  0.019125  0.018892  0.018565   
Fukuoka   Kitakyushu kyushu    0.024560  0.024827  0.025364  0.025496   
          Fukuoka    kyushu    0.032936  0.031408  0.029588  0.028649   
Kumamoto  Kumamoto   kyushu    0.015672  0.015513  0.014753  0.014041   
Miyagi    Sendai     tohoku    0.023016  0.022804  0.022075  0.021474   
Niigata   Niigata    tohoku    0.015377  0.014729  0.013474  0.013490   
Saitama   Saitama    tokyo     0.007481  0.007088  0.006809  0.006874   
Chiba     Chiba      tokyo     0.004989  0.004762  0.004818  0.005077   
Kanagawa  Yokohama   tokyo     0.021007  0.021027  0.020617  0.020195   
          Kawasaki   tokyo     0.013206  0.013643  0.014266  0.014041   
          Sagamihara tokyo     0.004421  0.004467  0.004496  0.004571   
Tokyo     Tokyo      tokyo     0.066072  0.064607  0.061969  0.060533   

                                2022.12    2023.1    2023.2    2023.3  \
pref      muni       epco                                               
Shizuoka  Shizuoka   chubu     0.008357  0.008937  0.008606  0.008191   
          Hamamatsu  chubu     0.010857  0.011372  0.011261  0.010952   
Aichi     Nagoya     chubu     0.024143  0.026490  0.026108  0.025276   
Okayama   Okayama    chugoku   0.016802  0.017660  0.017692  0.016920   
Hiroshima Hiroshima  chugoku   0.027287  0.029085  0.029081  0.027296   
Hokkaido  Sapporo    hokkaido  0.114798  0.117121  0.117338  0.113343   
Kyoto     Kyoto      kansai    0.019413  0.020309  0.020148  0.019293   
Osaka     Osaka 

In [15]:
# ============================================================================ #
# Calculate the residential energy demand of each main cities
# ============================================================================ #

main_city_resi_elec_dmd = []
for index, row in main_city_epco_elec_resi_share.iterrows():
    epco_name = index[2]
    muni_name = index[1]
    muni_resi_share = row["total"]
    main_city_resi_elec_dmd.append(
        (epco_hourly_demand_fy2022_scaled[epco_name] * muni_resi_share).to_frame(muni_name)
    )
main_city_resi_elec_dmd = pd.concat(main_city_resi_elec_dmd, axis=1)
main_city_resi_elec_dmd.to_csv("data/gen/major_city_resi_elec_dmd.csv")
main_city_resi_elec_dmd.rename(columns=location_jp_to_en)

,Shizuoka,Hamamatsu,Nagoya,Okayama,Hiroshima,Sapporo,Kyoto,Osaka,Sakai,Kobe,...,Fukuoka,Kumamoto,Sendai,Niigata,Saitama,Chiba,Yokohama,Kawasaki,Sagamihara,Tokyo
2022-04-01 00:00:00,101.496396,130.301193,303.894762,94.320273,149.592693,343.444752,238.277826,580.727608,176.463515,230.974837,...,227.435909,110.963142,187.070145,114.074516,173.007338,117.107569,493.974738,329.918252,108.453990,1512.948664
2022-04-01 01:00:00,99.462946,127.690647,297.806320,95.114788,150.852800,359.545359,232.677122,567.077645,172.315752,225.545789,...,226.718257,110.613009,192.357638,117.298805,167.124130,113.125263,477.176862,318.699205,104.765954,1461.500034
2022-04-01 02:00:00,101.072094,129.756473,302.624339,99.946736,158.516307,371.463989,242.262589,590.439221,179.414546,234.837471,...,239.665897,116.930001,199.206837,121.475415,167.945043,113.680934,479.520752,320.264654,105.280564,1468.678913
2022-04-01 03:00:00,103.017481,132.253964,308.449109,106.870366,169.497239,384.219015,251.058449,611.876376,185.928574,243.363746,...,258.025830,125.887583,204.895911,124.944586,171.502331,116.088839,489.677607,327.048264,107.510539,1499.787387
2022-04-01 04:00:00,103.353721,132.685629,309.455859,109.075550,172.994679,391.223825,252.288768,614.874893,186.839722,244.556356,...,266.577851,130.060007,209.982613,128.046433,175.743713,118.959804,501.787703,335.136414,110.169356,1536.878259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31 19:00:00,109.998461,141.216153,329.351162,95.098573,150.827083,351.599605,269.164332,656.003798,199.337408,260.914701,...,269.538166,131.504308,188.007169,114.645909,204.886111,138.686109,584.995785,390.709832,128.438000,1791.728450
2023-03-31 20:00:00,106.972302,137.331168,320.290409,92.974462,147.458226,338.426382,260.625554,635.193198,193.013769,252.637628,...,260.358199,127.025517,181.403381,110.618949,198.250401,134.194438,566.049344,378.055791,124.278238,1733.699181
2023-03-31 21:00:00,104.322412,133.929236,312.356258,90.299055,143.215009,322.221226,251.003360,611.742114,185.887776,243.310345,...,249.862537,121.904814,172.858615,105.408390,188.741496,127.757921,538.899289,359.922680,118.317343,1650.543837
2023-03-31 22:00:00,101.344288,130.105916,303.439327,86.926420,137.865984,309.361651,239.838678,584.531696,177.619449,232.487850,...,243.882103,118.987035,165.295493,100.796433,180.532370,122.201216,515.460393,344.268196,113.171246,1578.755051
